In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=371698bf5e266223264f4411c287377bad38d4d15663e4765c53d1180a7d0d74
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=37c10c5acc7e0f52a22289fb18699413db8c3076a25d5641810f8c8893b59c97
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images, test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def bulid_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
     filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
     kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
     activation='relu',
     input_shape=(28,28,1)
 ),
 keras.layers.Conv2D(
     filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
     kernel_size=hp.Choice('conv_2_kernal', values=[3,5]),
     activation='relu'
 ),
 keras.layers.Flatten(),
 keras.layers.Dense(
     units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
     activation='relu'
 ),
  keras.layers.Dense(10, activation='softmax')
 ])
 
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy']) 
 
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch (bulid_model,
                           objective='val_accuracy',
                           max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 36m 22s]
val_accuracy: 0.909166693687439

Best val_accuracy So Far: 0.9138333201408386
Total elapsed time: 01h 49m 16s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                3539040   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,596,330
Trainable params: 3,596,330
Non-trainable params: 0
_________________________________________________________________
